In [1]:
%load_ext autoreload
%autoreload 2
import torch
from functools import partial
import os
import sys
from ray import tune
from ray.train import Checkpoint, session
from ray.tune.schedulers import ASHAScheduler
from src import tokenize
from src import utils
from CustomDatasets import SimpleDataset
from Models import SimpleModel
import numpy
import random

2024-02-21 17:20:21,270	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-02-21 17:20:22,155	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
vocab = tokenize.readVocabulary("vocab.csv")
data = tokenize.readData("CalibratorTestingMultipleFiles.csv")
preTokens = tokenize.getPreTokens(data)
tokens = tokenize.tokenizePreTokens(random.sample(preTokens, 1000), vocab, 100, tokenize.TokenFormat.TwoDimensional)
train, test = utils.splitData(tokens, 0.9, 0.1)

C:\Users\elabo\Documents\GitHub\RetentionTimeEstimators\src\tokenize.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sequence = str(row[0])
C:\Users\elabo\Documents\GitHub\RetentionTimeEstimators\src\tokenize.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  preTokens.append((sequence, row[1]))
C:\Users\elabo\Documents\GitHub\RetentionTimeEstimators\src\tokenize.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  preTokens.

In [3]:
"training data: "+str(len(train)), " | testing data: "+str(len(test)), " | total: "+str(len(train)+len(test))

('training data: 720', ' | testing data: 80', ' | total: 800')

In [4]:
trainingSequences = []
trainingRetentionTimes = []
for i in train:
    trainingSequences.append(i[0])
    trainingRetentionTimes.append(i[1])

testingSequences = []
testingRetentionTimes = []
for i in train:
    testingSequences.append(i[0])
    testingRetentionTimes.append(i[1])

In [5]:
trainingSequences = numpy.stack(trainingSequences)
trainingRetentionTimes = numpy.stack(trainingRetentionTimes)
testingSequences = numpy.stack(testingSequences)
testingRetentionTimes = numpy.stack(testingRetentionTimes)

In [6]:
trainingDataset = SimpleDataset.PeptidesWithRetentionTimes(trainingSequences, trainingRetentionTimes)
testingDataset = SimpleDataset.PeptidesWithRetentionTimes(testingSequences, testingRetentionTimes)

In [7]:
f, rt = trainingDataset[5]
f, rt

(array([[ 1,  8,  4, 10, 16, 10,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0]]),
 0.2576699483882228)

In [8]:
a = torch.utils.data.DataLoader(trainingDataset, 64)
for i, a in a:
    print(i.shape)
    break

torch.Size([64, 2, 100])


In [9]:
# trainingDataset = trainingDataset
# testingDataset = testingDataset

In [10]:
# trainingDataloader = torch.utils.data.DataLoader(trainingDataset, batch_size=32, shuffle=True, drop_last=True)
# testingDataloader = torch.utils.data.DataLoader(testingDataset, batch_size=32, shuffle=True, drop_last=True)

In [11]:
class TunableAttentionRegression(torch.nn.Module):
    def __init__(self, input_size = 2707, embedding_size = 32, hidden_size = 512, output_size = 1, numberOfHeads = 1) -> None:
        super(AttentionRegression, self).__init__()
        self.embedding = torch.nn.Embedding(input_size, 32)
        self.lstm = torch.nn.LSTM(32, hidden_size, batch_first=True)
        self.attention = torch.nn.MultiheadAttention(hidden_size, num_heads=numberOfHeads)
        self.fc = torch.nn.Linear(hidden_size, output_size)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x) -> torch.Tensor:
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = lstm_out.permute(1, 0, 2)  # [seq_len, batch, hidden_size]
        attention_output, _ = self.attention(lstm_out, lstm_out, lstm_out)
        output = self.fc(attention_output.mean(dim=0))
        output = self.sigmoid(output)
        return output

In [19]:
def train_model(config, trainSet, testingSet):
    model = TunableAttentionRegression(int(config["input_size"]), config["embedding_size"], config["hidden_size"],
                                   config["output_size"], config["numberOfHeads"])
    criterion = torch.nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

    checkpoint = session.get_checkpoint()
    
    if checkpoint:
        checkpoint_state = checkpoint.to_dict()
        start_epoch = checkpoint_state["epoch"]
        model.load_state_dict(checkpoint_state["net_state_dict"])
        optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
    else:
        start_epoch = 0
    
    trainLoader = torch.utils.data.DataLoader(
           trainSet, batch_size=32, shuffle=True, num_workers=1)
    validationLoader = torch.utils.data.DataLoader(
        testingSet, batch_size=32, shuffle=True, num_workers=1)
    
    for epoch in range(start_epoch, 10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainLoader):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data[0]
            labels = data[1]
            # zero the parameter gradients
            optimizer.zero_grad()
    
            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print(
                    "[%d, %5d] loss: %.3f"
                    % (epoch + 1, i + 1, running_loss / epoch_steps)
                )
                running_loss = 0.0
    
        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(validationLoader, 0):
            with torch.no_grad():
                inputs, labels = data
    
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1
    
        checkpoint_data = {
            "epoch": epoch,
            "net_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        }
        checkpoint = Checkpoint.from_dict(checkpoint_data)
    
        session.report(
            {"loss": val_loss / val_steps, "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")
    

In [20]:
def test_accuracy(model, trainset, testset):
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            input, labels = data
            outputs = model(input)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [21]:
# config = {
#     "input_size": 2707,
#     "embedding_size": tune.choice([2 ** i for i in range(9)]),
#     "hidden_size": tune.choice([2 ** i for i in range(9)]),
#     "output_size": 1,
#     "numberOfHeads": tune.choice([1 + i for i in range(9)]),
#     "lr": tune.loguniform(1e-4, 1e-1),
#     "batch_size": 16
# }

In [22]:
def main(num_samples=10, max_num_epochs=10):
    config = {
        "input_size": 2707,
        "embedding_size": tune.choice([2 ** i for i in range(9)]),
        "hidden_size": tune.choice([2 ** i for i in range(9)]),
        "output_size": 1,
        "numberOfHeads": tune.choice([1 + i for i in range(9)]),
        "lr": tune.loguniform(1e-4, 1e-1)}
    
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    
    result = tune.run(
        partial(train_model, trainingDataset, testingDataset),
        resources_per_trial={"cpu": 1},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler)

    best_trial = result.get_best_trial("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

    best_trained_model = TunableAttentionRegression(best_trial.config["input_size"], best_trial.config["embedding_size"],
                              best_trial.config["hidden_size"], best_trial.config["output_size"],
                              best_trial.config["numberOfHeads"])
    

    best_checkpoint = best_trial.checkpoint.to_air_checkpoint()
    best_checkpoint_data = best_checkpoint.to_dict()

    best_trained_model.load_state_dict(best_checkpoint_data["net_state_dict"])

    test_acc = test_accuracy(best_trained_model, traininDataset, testingDataset)
    print("Best trial test set accuracy: {}".format(test_acc))


In [23]:
if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=10)

2024-02-21 17:22:37,077	INFO tune.py:583 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2024-02-21 17:22:46,766	ERROR tune_controller.py:1374 -- Trial task failed for trial train_model_19680_00003
Traceback (most recent call last):
  File "C:\Users\elabo\anaconda3\envs\RTPy3.9\lib\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "C:\Users\elabo\anaconda3\envs\RTPy3.9\lib\site-packages\ray\_private\auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\elabo\anaconda3\envs\RTPy3.9\lib\site-packages\ray\_private\client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\elabo\anaconda3\envs\RTPy3.9\lib\site-packages\ray\_private\worker.py", line 2624, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(ValueError): ray::ImplicitFunc.train() (pid=37408, ip=127.0.0.1, actor_id=ff7731ba5e02faa4d1d6e47401000000, repr=func)
  File "python\ray\_raylet.pyx", line 1813, in ray._raylet.execute_task
  File "pytho

Trial name
train_model_19680_00000
train_model_19680_00001
train_model_19680_00002
train_model_19680_00003
train_model_19680_00004
train_model_19680_00005
train_model_19680_00006
train_model_19680_00007
train_model_19680_00008
train_model_19680_00009


2024-02-21 17:22:46,878	ERROR tune_controller.py:1374 -- Trial task failed for trial train_model_19680_00001
Traceback (most recent call last):
  File "C:\Users\elabo\anaconda3\envs\RTPy3.9\lib\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "C:\Users\elabo\anaconda3\envs\RTPy3.9\lib\site-packages\ray\_private\auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\elabo\anaconda3\envs\RTPy3.9\lib\site-packages\ray\_private\client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\elabo\anaconda3\envs\RTPy3.9\lib\site-packages\ray\_private\worker.py", line 2624, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(ValueError): ray::ImplicitFunc.train() (pid=42904, ip=127.0.0.1, actor_id=d0a07bdadd2feb2d0d51ec6701000000, repr=func)
  File "python\ray\_raylet.pyx", line 1813, in ray._raylet.execute_task
  File "pytho

TuneError: ('Trials did not complete', [train_model_19680_00000, train_model_19680_00001, train_model_19680_00002, train_model_19680_00003, train_model_19680_00004, train_model_19680_00005, train_model_19680_00006, train_model_19680_00007, train_model_19680_00008, train_model_19680_00009])